# Topic Modeling

## Import data and Library 

In [0]:
import nltk
import gensim
from keras.utils.data_utils import get_file
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim 
import io
import re

Using TensorFlow backend.


In [0]:
path = get_file("debatcapres_4000.txt", origin="https://transfer.sh/GUMbI/debatcapres_4000.txt")
with io.open(path, encoding="utf-8") as f:
  text = f.read().lower()
print(len(text))

25297


## Preposessing for Topic Modelling
Melakukan Proses preposeseng yang diperlukan untuk proses selanjutnya

In [0]:
# Konversikan dokumen menjadi list of token dengan lower case, dan mengabaikan token 
# yang terlalu pendek atau terlalu panjang.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def split_into_tokens(text):
    tokens = nltk.word_tokenize(text)
    return tokens

In [0]:
def preprocessing(text):    
    # Remove punctuation & special characters
    tokens = re.sub('[^ a-zA-Z0-9]', '', text)
    
    # Tokenize
    tokens = split_into_tokens(tokens)    
    
    # Remove stopwords
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop]
    
    preprocessed_text = tokens
    return preprocessed_text 

In [0]:
import gensim
from gensim import corpora

filtered_sentence = preprocessing(text)
tokenized_sents = list(sent_to_words(filtered_sentence))

# Membuat term dictionary dari corpus kita, dimana setiap unique term diberikan sebuah index. 
dictionary = corpora.Dictionary(tokenized_sents)


# Merubah list documents (corpus) ke Document Term Matrix dengan dictionary yang above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_sents]

## Creating the object for LDA model & train the model
Running and Training LDA model di document term matrix dengan memilih minimum parameters yang dibutuhkan.

In [0]:
Lda = gensim.models.ldamodel.LdaModel

ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, alpha='auto', eval_every=5, passes=100)

## Extract three topics with twenty words in each topic
Menampilkan tiga topic yang sudah di generate dengan masing masing 20 kata untuk tiap topic

In [0]:
print(ldamodel.print_topics(num_topics=3, num_words=30))

[(0, '0.060*"sandi" + 0.043*"want" + 0.039*"people" + 0.021*"give" + 0.017*"indonesia" + 0.015*"prices" + 0.014*"go" + 0.012*"money" + 0.012*"time" + 0.011*"better" + 0.010*"economy" + 0.010*"must" + 0.009*"believe" + 0.009*"country" + 0.009*"nation" + 0.008*"community" + 0.007*"home" + 0.007*"create" + 0.006*"stop" + 0.006*"much" + 0.005*"says" + 0.005*"willing" + 0.005*"harvest" + 0.005*"easily" + 0.005*"rice" + 0.005*"prosperous" + 0.005*"status" + 0.005*"development" + 0.005*"course" + 0.005*"strategy"'), (1, '0.027*"wants" + 0.025*"get" + 0.024*"us" + 0.023*"thank" + 0.020*"also" + 0.018*"one" + 0.018*"work" + 0.011*"strong" + 0.011*"able" + 0.010*"future" + 0.010*"honorary" + 0.010*"economic" + 0.009*"jobs" + 0.008*"ready" + 0.008*"president" + 0.008*"make" + 0.007*"farmers" + 0.007*"cheap" + 0.007*"food" + 0.007*"children" + 0.007*"together" + 0.007*"given" + 0.006*"bumn" + 0.006*"many" + 0.006*"electricity" + 0.006*"women" + 0.006*"launched" + 0.006*"right" + 0.006*"actually" +

## Check Perplexity and Compute Coherence Score
* Mengecek seberapa baik model yang telah dibuat. Semakin  rendah Perplexity maka semaik baik.
* mengecek nilai keterkaitan dari tiap topik yang di generete. Semakin  tinggi Coherence Score maka semaik baik.

In [0]:
print('\nPerplexity: ', ldamodel.log_perplexity(doc_term_matrix))

coherence_model_lda = CoherenceModel(model=ldamodel, texts=tokenized_sents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.787542168676612

Coherence Score:  0.8526228778539537


## Visualize the topics model
pyLDAvis tool unutk memvisualisasikan kesesuaian dari LDA model kita berdasarkan topics dan top words nya. 

In [0]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\Haris\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=             x         y  topics  cluster       Freq
topic                                               
2     -0.19591 -0.008977       1        1  39.379070
1      0.10648 -0.150223       2        1  31.293121
0      0.08943  0.159200       3        1  29.327816, topic_info=    Category       Freq         Term      Total  loglift  logprob
132  Default  39.000000        sandi  39.000000  30.0000  30.0000
60   Default  49.000000      prabowo  49.000000  29.0000  29.0000
283  Default  28.000000         want  28.000000  28.0000  28.0000
26   Default  25.000000       people  25.000000  27.0000  27.0000
837  Default  19.000000        wants  19.000000  26.0000  26.0000
285  Default  18.000000          get  18.000000  25.0000  25.0000
232  Default  17.000000           us  17.000000  24.0000  24.0000
65   Default  16.000000        thank  16.000000  23.0000  23.0000
475  Default  14.000000         give  14.000000  22.0000  22.0000
277  Default  14.000000         also  14.000000  21.0000  21.0000
31   Default  13.000000          one  13.000000  20.0000  20.0000
97   Default  12.000000         work  12.000000  19.0000  19.0000
166  Default  11.000000    indonesia  11.000000  18.0000  18.0000
969  Default  15.000000     puncture  15.000000  17.0000  17.0000
123  Default  10.000000       prices  10.000000  16.0000  16.0000
233  Default   9.000000           go   9.000000  15.0000  15.0000
83   Default  13.000000        young  13.000000  14.0000  14.0000
967  Default  12.000000          tps  12.000000  13.0000  13.0000
457  Default   8.000000         time   8.000000  12.0000  12.0000
47   Default   8.000000        money   8.000000  11.0000  11.0000
301  Default   7.000000       better   7.000000  10.0000  10.0000
66   Default  10.000000          god  10.000000   9.0000   9.0000
198  Default   8.000000       strong   8.000000   8.0000   8.0000
332  Default   8.000000         able   8.000000   7.0000   7.0000
58   Default   7.000000         must   7.000000   6.0000   6.0000
87   Default   7.000000      economy   7.000000   5.0000   5.0000
603  Default   7.000000       future   7.000000   4.0000   4.0000
983  Default   9.000000      skewers   9.000000   3.0000   3.0000
623  Default   7.000000     honorary   7.000000   2.0000   2.0000
6    Default   7.000000     economic   7.000000   1.0000   1.0000
..       ...        ...          ...        ...      ...      ...
132   Topic3  39.198307        sandi  39.674885   1.2145  -2.8122
283   Topic3  27.914141         want  28.390371   1.2097  -3.1517
26    Topic3  25.509184       people  25.984093   1.2082  -3.2418
475   Topic3  13.692535         give  14.169080   1.1924  -3.8640
166   Topic3  11.355796    indonesia  11.830712   1.1857  -4.0511
123   Topic3   9.887086       prices  10.362322   1.1797  -4.1896
233   Topic3   9.152655           go   9.628127   1.1760  -4.2668
457   Topic3   8.050841         time   8.526832   1.1692  -4.3951
47    Topic3   8.050841        money   8.526832   1.1692  -4.3951
301   Topic3   7.316121       better   7.792631   1.1635  -4.4908
58    Topic3   6.814874         must   7.289745   1.1593  -4.5618
87    Topic3   6.814874      economy   7.289745   1.1593  -4.5618
299   Topic3   6.148351      believe   6.623456   1.1522  -4.6647
39    Topic3   6.148351      country   6.623456   1.1522  -4.6647
5     Topic3   5.781120       nation   6.256358   1.1476  -4.7263
94    Topic3   5.413855    community   5.889258   1.1425  -4.7919
276   Topic3   4.679167         home   5.155059   1.1298  -4.9378
240   Topic3   4.612026       create   5.087165   1.1286  -4.9522
721   Topic3   3.944091         much   4.420855   1.1125  -5.1087
176   Topic3   3.944091         stop   4.420855   1.1125  -5.1087
188   Topic3   3.576268       easily   4.053748   1.1013  -5.2066
363   Topic3   3.576268         says   4.053748   1.1013  -5.2066
733   Topic3   3.576268      harvest   4.053748   1.1013  -5.2066
931   Topic3   3.576268   prosperous   4.053748   1.1013  -5.2